<a href="https://colab.research.google.com/github/mdorodch/ITSC3155-Demos/blob/master/Hour3_policy_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/workshop_RLHF

Mounted at /content/gdrive
/content/gdrive/MyDrive/workshop_RLHF


In [ ]:
import numpy as np
from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical #Creates a categorical distribution parameterized by input

In [ ]:
m = Categorical(torch.tensor([ 0.25, 0.25, 0.25, 0.25 ]))
m.sample()

tensor(0)

### The CartPole-v1 environment

The episode ends if: The pole Angle is greater than ±12° ; Cart Position is greater than ±2.4; Episode length is greater than 500

We get a reward +1 every timestep the Pole stays in the equilibrium.

In [ ]:
import gym
env = gym.make("CartPole-v1")
eval_env = gym.make("CartPole-v1") # Create the evaluation env

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
print(env.observation_space)
print(env.action_space)
# Get the state space and action space
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print(state_size, action_size)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Discrete(2)
4 2


In [ ]:
print("Sample state", env.observation_space.sample()) # Get a random state
print("Action Sample", env.action_space.sample()) # Take a random action

Sample state [ 2.4505110e+00 -1.3210601e+38 -2.2125986e-01  4.2570131e+37]
Action Sample 1


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
a=[1, 2,4]
a.unsqueeze(0)
[[1][2][4]]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: ignored

In [ ]:
#build a neural network that is a policy function
#Two fully connected layers (fc1 and fc2).
#Using ReLU as activation function of fc1
#Using Softmax to output a probability distribution over actions
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()  #sample a random action from current distribution
        return action.item(), m.log_prob(action)

we can't use `action = np.argmax(m)` since it will always output the action that have the highest probability. We need to replace with `action = m.sample()` that will sample an action from the probability distribution P(.|s)

In [ ]:
# Create policy and place it to the device
h_size= 16
lr=1e-2
myPolicy = Policy(state_size, action_size, h_size).to(device)
optimizer = optim.Adam(myPolicy.parameters(), lr=lr)

## Use REINFORCE training method

In [ ]:
#Hyper parameters


In [ ]:
def reinforce(policy, optimizer):
    episodes= 400
    max_t= 1000
    gamma= 1.0 #when this is 1, there is no discount

    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100) #double-ended queue
    scores = []

    for episode in range(episodes):
        saved_log_probs = []
        rewards = []
        state = env.reset()

        #take multiple period's actions and gather reward
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        ##  "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        returns = deque(maxlen=max_t)
        n_steps = len(rewards) #rewards is an array that holds trials
        for t in range(n_steps)[::-1]: #reversed list, [t-1, t-2, ..., 0], the last item is 0
            #E.g. print(range(5)[::-1])  will generate range(4, -1, -1)
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(gamma*disc_return_t + rewards[t]   )

        ## standardization of the returns is employed to make training more stable
        eps = np.finfo(np.float32).eps.item()  ## eps is the smallest representable float, added to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        # gradient descent
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()  #update the weight

        if episode % 100 == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_deque)))

    return scores

In [ ]:
scores = reinforce(myPolicy,optimizer)

## Define evaluation method


In [ ]:
eval_episodes= 10
def evaluate_agent(env, max_steps, eval_episodes, policy):
  """
  Evaluate the agent for ``eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param eval_episodes: Number of episode to evaluate the agent
  :param policy: The Reinforce agent
  """
  episode_rewards = []
  for episode in range(eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward

      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

In [ ]:
evaluate_agent(eval_env,max_t,eval_episodes,myPolicy)

(100.0, 0.0)

In [ ]:
import imageio

def record_video(env, policy, out_directory, fps=30):
  images = []
  done = False
  state = env.reset()
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # Take the action (index) that have the maximum expected future reward given that state
    action, _ = policy.act(state)
    state, reward, done, info = env.step(action) # We directly put next_state = state for recording logic
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
# Record and save a video
video_path =  "replay1.mp4"
video_fps=60
record_video(env, myPolicy, video_path, video_fps)